In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F14872%2F228180%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240912%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240912T144028Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D92a0c706efba37162064e16dc8680ed6315c41eb7a8227332d691094fe38e1f9ebacd111a91190c74fe7dae63c0024d933017c5075e118f1703226a4b7191e40c92dbddbc2ffa8626484d59ce6affaa170eef0175cfc61883d318ee75445b718be3a970142061e8e506511f733a560a5a3b2f23766b9b1e6c4c0a391939777acc37226e4a2bb198b2cdf848a65750c1010bc7160246c66e8ad40891da26be4d199d0cea6b24f55223aafc74c013f1389eed24420ab4fa86c0f990e75a8b6c11f925111a76582aecd7f4f1b0ede24706e41fcae3ba4165ccf8884e8d99d912b6a62921aa3b44dcdf0a2e257c0ed84a606d20809be672f089827b38f022dcb0348'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 9867 bytes downloaded
Downloaded and uncompressed: 
Data source import complete.


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import mean_absolute_error

In [4]:
dataset = pd.read_csv('../input/Admission_Predict_Ver1.1.csv')

In [5]:
dataset.info()
dataset.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          500 non-null    int64  
 2   TOEFL Score        500 non-null    int64  
 3   University Rating  500 non-null    int64  
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 35.3 KB


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.472000,107.192000,3.114000,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.295148,6.081868,1.143512,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


There are no null values in the dataset ! Let's remove the Serial No column and check the correlation amongst the variables

In [ ]:
dataset = dataset.drop('Serial No.', axis=1)
sns.pairplot(data=dataset, kind='reg')

It is clearly visible that GRE score, TOEFL score, CGPA have heavy influence on the chances of admission. Research doesn't seem to have high infuence on the chances of admission. Let's try to check the correlation values

In [ ]:
dataset.corr(method='pearson')

Above data confirms the understanding. Also there exist multicollinearity in the data. GRE, CGPA, and TOEFL scores are heavily correlated.
Let's try to build the model with all independent variables first.

In [ ]:
y=dataset['Chance of Admit ']
dataset=dataset.drop('Chance of Admit ',axis=1)

def create_stats_fit_model(x):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=42)
    regressor_OLS = sm.OLS(endog=y_train,exog=x_train).fit()
    print(regressor_OLS.summary())
    predictions =  regressor_OLS.predict(x_test)
    print("Mean absolute error is ",mean_absolute_error(y_test,predictions))

Let's now call the method with full set of independent variables

In [ ]:
dataset_temp = np.append(arr = np.ones((500,1)).astype(int), values = dataset ,axis=1)
dataset_temp1 = dataset_temp[:, [0,1,2,3,4,5,6,7]]
create_stats_fit_model(dataset_temp1)

Let's remove x4 which has the highest pvalue and rerun

In [ ]:
dataset_temp1 = dataset_temp[:, [0,1,2,3,5,6,7]]
create_stats_fit_model(dataset_temp1)

Let's remove x3 and rerun

In [ ]:
dataset_temp1 = dataset_temp[:, [0,1,2,5,6,7]]
create_stats_fit_model(dataset_temp1)


There is no observable improvement further to this model. It seems that as per my analysis, I would
consider below variables -
GRE Score, TOEFL Score, LOR, CGPA, Research


I still think that this could be improved as have not considered the impact of multicollinearity. Please comment on what do you think.